In [1]:
import sympy as sym
import numpy as np
from tqdm import tqdm

In [2]:
# define parameters
P = {
    'a=0': sym.symbols('p_1'),
    'b=0|a=0': sym.symbols('p_2'),
    'b=0|a=1': sym.symbols('p_3'),
    'c=0|b=0': sym.symbols('p_4'),
    'c=0|b=1': sym.symbols('p_5')
} 

In [3]:
# define all marginal proba
Margins = {
    'a=0': P['a=0'],
    'a=1': 1 - P['a=0']
} 

Margins['b=0'] = P['b=0|a=0']*Margins['a=0'] + P['b=0|a=1']*Margins['a=1']
Margins['b=1'] = 1 - Margins['b=0']
Margins['c=0'] = P['c=0|b=0']*Margins['b=0'] + P['c=0|b=1']*Margins['b=1']
Margins['c=1'] = 1 - Margins['c=0']

In [4]:
Joint_proba = {
    'a=0,b=0': P['b=0|a=0'] * P['a=0'],
    'a=1,b=1': (1 - P['b=0|a=1']) * Margins['a=1'],
    'b=0,c=0': P['c=0|b=0'] * Margins['b=0'],
    'b=1,c=1': (1 - P['c=0|b=1']) * Margins['b=1'],
    'a=0,b=0,c=0': P['a=0']*P['b=0|a=0']*P['c=0|b=0'],
    'a=1,b=1,c=1': (1-P['a=0'])*(1-P['b=0|a=1'])*(1-P['c=0|b=1'])
}
BN_estimations = {
    'a=b': Joint_proba['a=0,b=0'] + Joint_proba['a=1,b=1'],
    'b=c': Joint_proba['b=0,c=0'] + Joint_proba['b=1,c=1'],
    'a=b=c': Joint_proba['a=0,b=0,c=0'] + Joint_proba['a=1,b=1,c=1'],
}

In [5]:
# Simulate the estimation with independent assumption
Independent_estiamtions = {
    'a=b': Margins['b=0'] * Margins['a=0'] + Margins['b=1'] * Margins['a=1'],
    'b=c': Margins['b=0'] * Margins['c=0'] + Margins['b=1'] * Margins['c=1'],   
    'a=b=c': Margins['a=0'] * Margins['b=0'] * Margins['c=0'] + Margins['a=1'] * Margins['b=1'] * Margins['c=1']
}

In [6]:
values = {'p_1': 0.6, 'p_2': 0.6, 'p_3': 0, 'p_4': 0.8, 'p_5': 0.1,}
for q in BN_estimations:
    BN_est = BN_estimations[q].evalf(subs=values)
    avi_est = Independent_estiamtions[q].evalf(subs=values)
    print(f"For query P({q})")
    print(f"\tBN_est = {BN_est:.2f}, Indep_est = {avi_est:.2f}")    

For query P(a=b)
	BN_est = 0.76, Indep_est = 0.47
For query P(b=c)
	BN_est = 0.86, Indep_est = 0.54
For query P(a=b=c)
	BN_est = 0.65, Indep_est = 0.24


In [12]:
# with dependency/correlation
repeat = 100
under_est_cnt = 0
for r in range(repeat):
    for q in BN_estimations:
        r = np.random.rand(5)
        values['p_1'] = r[0]
        values['p_2'] = r[1] if r[1] >= 0.5 else 1-r[1]
        values['p_3'] = r[2] if r[2] <= 0.5 else 1-r[2]
        values['p_4'] = r[3] if r[3] >= 0.5 else 1-r[3]
        values['p_5'] = r[4] if r[4] <= 0.5 else 1-r[4]
        BN_est = BN_estimations[q].evalf(subs=values)
        avi_est = Independent_estiamtions[q].evalf(subs=values)
        under_est_cnt += (1 if avi_est < BN_est else 0)
print(f"{under_est_cnt/3}/{repeat} are underestimations")

100.0/100 are underestimations
